## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!
import sqlite3
import pandas as pd

In [3]:
df=pd.read_csv('zippedData/bom.movie_gross.csv.gz')
print(df.head())


                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  


### BUSINESS UNDERSTANDING

##### KEY QUESTIONS FOR FOR MOVIE STUDIO STRATEGY
1. Which movie genres generate the highest revenue- by analyzing the total earnings fromm the different genres, we can determine which types of films are more profitable and likely to attact large audiences.
2. Does movie rating or critic score affect the performance- using ratings from Rotten tomatoes and movies data base , we can measure whether higher rated movies earn more revenue 
3. Does a longer runtime have an effect on rating or revenue- by analyzing the runtime in the dataset, we are able to identify which runtime generate higher earnings for different types of movies.
4. Does the release timing affect the earnings- by examining which months and their associated revene, we can identify the optimal times to launch fils for maximum success